THIS SCRIPT PREPROCESSES THE CONSOLIDATED DATA COLLECTED AND CREATES SEPERATE TABLES FOR ANALYSIS

In [2]:
# Dependencies
from config import homeDepot_API
from serpapi import GoogleSearch
import pandas as pd
import numpy as np
from openpyxl import load_workbook

STEP 1: PREPROCESS DATA

In [21]:
# Collect all data

path0 = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiHistoricalPrices.csv"
raw_prices = pd.read_csv(path0)

path = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\data_final.csv"
data = pd.read_csv(path)

print(data.shape, data.dtypes)

(1094, 9) material_description     object
vendor                   object
material_price          float64
link                     object
formatted_datetime       object
dow                      object
vendor_id               float64
location                 object
product_id              float64
dtype: object


In [22]:
data.count()

material_description    1094
vendor                  1094
material_price          1094
link                    1094
formatted_datetime      1094
dow                     1094
vendor_id               1085
location                1085
product_id              1044
dtype: int64

In [23]:
data.drop(columns =['vendor_id', 'product_id'], inplace = True)
data.dropna(inplace=True)

In [24]:
data.count()

material_description    1085
vendor                  1085
material_price          1085
link                    1085
formatted_datetime      1085
dow                     1085
location                1085
dtype: int64

In [25]:
# Create Regional columns

# Split location column to create a zip code column

data['zip'] = (np.where(data['location'].str.contains(','), data['location'].str.split(',').str[-1], data['location']))
data['zip'] = data['zip'].str.strip()
data['zip'] = (np.where(data['zip'].str.len()>5, "other", data['zip']))

region = []
for i in data['zip']:
    if i.startswith("0"):
        region.append('northeast')
    elif i.startswith("1"):
        region.append('northeast')
    elif i.startswith("2"):
        region.append('east')
    elif i.startswith("3"):
        region.append('southeast')
    elif i.startswith("4"):
        region.append('midwest')
    elif i.startswith("5"):
        region.append('northplaines')
    elif i.startswith("6"):
        region.append('central')
    elif i.startswith("7"):
        region.append('south')
    elif i.startswith("8"):
        region.append('mountains')
    elif i.startswith("9"):
        region.append('west')
    else:
        region.append('other')
        
        
data['region'] = region
data.formatted_datetime = pd.to_datetime(data.formatted_datetime)
data['date'] = data.formatted_datetime.dt.date
data.date = pd.to_datetime(data.date)
data.drop(columns =['formatted_datetime'], inplace = True)

In [26]:
raw = []
for i in data['material_description']:
    if i.startswith("1") or i.startswith("3"):
        raw.append('steel')    
    else:
        raw.append('copper')
        
        
data['raw_material'] = raw

In [27]:
data.dtypes

material_description            object
vendor                          object
material_price                 float64
link                            object
dow                             object
location                        object
zip                             object
region                          object
date                    datetime64[ns]
raw_material                    object
dtype: object

In [28]:
# Create raw prices features (NOTE: SOME HOW, I LOSE 100 PLUS ROWS OF DATA??)

raw_prices.date = pd.to_datetime(raw_prices.date)
raw_prices = raw_prices.iloc[:,[0,1,2]]
data = pd.merge(data, raw_prices)
data.shape

(991, 12)

In [29]:
# Creating 'material_description' by extracting from Home Depot. Home Depot's product id is what is used to query

material_name = []

ids = ['100400409', '100400406', '100137321', '100400405', 
       '202241090', '100144234', '202077145', '202077147', 
       '202288508', '202304774', '202304649', '202304645', 
       '202019377', '202316275', '202019375', '202316276']

for i in ids:

    params = {
      "engine": "home_depot_product",
      "product_id": i,
      "api_key": homeDepot_API
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = results['product_results']
    
    material_name.append(products['title'])
        
material_description = pd.DataFrame(data={'material_description': material_name}).iloc[[3,1,0,5,8,4,2,6,7,12,13,14,15,9,10,11],:].reset_index(drop=True)


https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [30]:
R = material_description['material_description']
R_list=R.tolist()
target_list = [[]]
for r in R_list:
    int_df = data.loc[data['material_description']==r] [['material_price']]
    Q1 = np.quantile(int_df, 0.25)
    Q3 = np.quantile(int_df, 0.75)
    data.loc[data['material_description']==r, 'target_price'] = np.where(int_df['material_price'] >= Q3, 'high price', (np.where(int_df['material_price'] <= Q1, 'low price', 'about average')))

data.head()

,material_description,vendor,material_price,link,dow,location,zip,region,date,raw_material,steel_price,copper_price,target_price
0,250 ft. 14/2 Romex SIMpull Solid NM-B W/G Wire,ebay,111.07,https://www.ebay.com/itm/363478081520,Saturday,"2025 Hamilton Avenue. San Jose, California, 95125",95125,west,2022-11-05,copper,1600.95,7953.85,about average
1,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...,amazon,98.99,https://www.amazon.com/EMT-Conduit-10-ft-Steel...,Saturday,"410 Terry Ave N. Seattle, WA, 98109",98109,west,2022-11-05,steel,1600.95,7953.85,high price
2,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,lowes,5.99,https://www.lowes.com/pd/Sigma-Electric-ProCon...,Saturday,"1000 Lowe's Blvd. Mooresville, NC, 28117",28117,east,2022-11-05,steel,1600.95,7953.85,high price
3,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,gordonelectricsupply,4.86,https://www.gordonelectricsupply.com/p/Allied-...,Saturday,"1290 N Hobbie Ave. Kankakee, Illinois, 60901",60901,central,2022-11-05,steel,1600.95,7953.85,low price
4,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,homedepot,7.22,https://www.homedepot.com/p/1-2-in-x-10-ft-Ele...,Saturday,"2455 Paces Ferry Rd SE. Atlanta, GA, 30339",30339,southeast,2022-11-05,steel,1600.95,7953.85,high price


In [34]:
descriptive_table = data.groupby('material_description').describe()['material_price']
descriptive_table = descriptive_table.drop('3/4 in. Electrical Metallic Tube (EMT) Set-Screw Connector (25-Pack)')
descriptive_table

,count,mean,std,min,25%,50%,75%,max
material_description,,,,,,,,
1 in. Electric Metallic Tube (EMT) Set-Screw Coupling,23.0,1.850000,1.740300,1.15,1.1500,1.15,1.1500,7.24
1 in. Electrical Metallic Tube (EMT) Set-Screw Connector,23.0,2.494783,3.954021,1.60,1.6000,1.60,1.6000,20.62
1 in. x 10 ft. Electric Metallic Tube (EMT) Conduit,59.0,19.805254,7.560292,5.99,17.5500,20.62,20.6200,34.23
1/2 in. Electric Metallic Tube (EMT) Set-Screw Coupling (5-Pack),52.0,7.565577,20.253125,2.10,3.5575,3.98,5.9900,150.00
1/2 in. Electrical Metallic Tube (EMT) Set-Screw Connectors (5-Pack),57.0,8.042807,19.256003,2.10,3.9800,5.99,5.9900,150.00
1/2 in. x 10 ft. Electric Metallic Tube (EMT) Conduit,83.0,8.490964,16.090482,4.59,5.8800,5.99,7.2200,150.00
250 ft. 12/2 Solid Romex SIMpull CU NM-B W/G Wire,49.0,155.932449,21.429853,111.07,149.0000,149.00,149.0000,227.29
250 ft. 12/3 Solid Romex SIMpull CU NM-B W/G Wire,57.0,289.629474,47.620473,84.99,289.0000,289.00,315.0100,339.09
250 ft. 14/2 Romex SIMpull Solid NM-B W/G Wire,85.0,104.330824,23.532501,2.98,90.0000,114.00,117.0000,128.62


In [35]:
# Display savings generated by using our Algorythm

avgYR= descriptive_table['mean'].sum()*365
minYR = descriptive_table['25%'].sum()*365
savings = avgYR - minYR

savings

102865.72398362705

In [36]:
descriptive_table = descriptive_table.transpose()
descriptive_table = descriptive_table.iloc[:,[5,12,2,3,11,0,4,10,1,8,9,6,7,14,15,13]]
descriptive_table.columns=mat_name = ['emt12','emt34','emt1','cup12','cup34','cup1','con12','con34','con1','rom142','rom143','rom122','rom123','wir4','wir6','wir10']
descriptive_table

,emt12,emt34,emt1,cup12,cup34,cup1,con12,con34,con1,rom142,rom143,rom122,rom123,wir4,wir6,wir10
count,83.000000,75.000000,59.000000,52.000000,67.000000,23.0000,57.000000,54.000000,23.000000,85.000000,66.000000,49.000000,57.000000,55.000000,138.000000,47.000000
mean,8.490964,23.330933,19.805254,7.565577,3.825224,1.8500,8.042807,7.671481,2.494783,104.330824,161.244394,155.932449,289.629474,774.473818,451.194058,177.109362
std,16.090482,31.609915,7.560292,20.253125,1.211926,1.7403,19.256003,4.767554,3.954021,23.532501,56.144943,21.429853,47.620473,180.294237,129.113728,68.799772
min,4.590000,8.120000,5.990000,2.100000,2.980000,1.1500,2.100000,4.600000,1.600000,2.980000,1.150000,111.070000,84.990000,613.640000,344.990000,145.000000
25%,5.880000,8.600000,17.550000,3.557500,2.980000,1.1500,3.980000,4.600000,1.600000,90.000000,158.000000,149.000000,289.000000,614.280000,409.000000,155.990000
50%,5.990000,10.470000,20.620000,3.980000,2.980000,1.1500,5.990000,5.990000,1.600000,114.000000,159.000000,149.000000,289.000000,709.000000,445.810000,155.990000
75%,7.220000,11.910000,20.620000,5.990000,3.660000,1.1500,5.990000,6.450000,1.600000,117.000000,160.192500,149.000000,315.010000,1047.780000,470.430000,165.000000
max,150.000000,98.990000,34.230000,150.000000,5.990000,7.2400,150.000000,20.000000,20.620000,128.620000,330.850000,227.290000,339.090000,1047.780000,1144.690000,398.950000


In [37]:
# Creating a dataframe for each material item for trouble shooting
new_df = []
for r in R:
    df = data[data['material_description']==r].reset_index(drop=True)    
    new_df.append(df)
test = pd.concat(new_df, axis=0, ignore_index=True)

In [38]:
preprocessed_df = data
preprocessed_df.shape

(991, 13)

In [39]:
# Send CSVs to my computer

preprocessed = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\preprocessed_df.csv"
preprocessed_df.to_csv(preprocessed, index=False)

descriptive = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\descriptive_table.csv"
descriptive_table.to_csv(descriptive, index=False)

emt12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\emt12.csv"
new_df[0].to_csv(emt12_df, index=False)

emt34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\emt34.csv"
new_df[1].to_csv(emt34_df, index=False)

emt1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\emt1.csv"
new_df[2].to_csv(emt1_df, index=False)

cup12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\cup12.csv"
new_df[3].to_csv(cup12_df, index=False)

cup34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\cup34.csv"
new_df[4].to_csv(cup34_df, index=False)

cup1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\cup1.csv"
new_df[5].to_csv(cup1_df, index=False)

con12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\con12.csv"
new_df[6].to_csv(con12_df, index=False)

con34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\con34.csv"
new_df[7].to_csv(con34_df, index=False)

con1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\con1.csv"
new_df[8].to_csv(con1_df, index=False)

rom142_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\rom142.csv"
new_df[9].to_csv(rom142_df, index=False)

rom143_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\rom143.csv"
new_df[10].to_csv(rom143_df, index=False)

rom122_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\rom122.csv"
new_df[11].to_csv(rom122_df, index=False)

rom123_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\rom123.csv"
new_df[12].to_csv(rom123_df, index=False)

wir4_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\wir4.csv"
new_df[13].to_csv(wir4_df, index=False)

wir6_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\wir6.csv"
new_df[14].to_csv(wir6_df, index=False)

wir10_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\AlexeiDataFrames\\wir10.csv"
new_df[15].to_csv(wir10_df, index=False)